This is a simple recommnedation system algorithm that using matrix factorization. The main point is to find a Mu = N_users x N_embed and a Mn = N_movies x N_embed matrices such that Ratings ~ Mu x Mm.T which is a N_user x N_movies matrix with element (i,j) being the rating provided by user i for movie j.  
I suggest to to look at this project by Google  
https://developers.google.com/machine-learning/recommendation  
to learn more about recommendation systems.  

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import torch.nn.functional as F
import torch
from torch import Tensor as TT
from torch import nn
import numpy as np
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from collections import OrderedDict
from glob import glob
import os
import os.path
import pathlib
from typing import Any, Callable, Optional, Sequence, Tuple, Union
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pandas as pd

Load the data containing user_id, movie_id, rating and timestamp. Rename the columns accordingly.

In [ ]:
mratings = pd.read_csv('/home/giangi/Workspace/Data/ml-100k/u.data', sep='\t', header=None)
columns=['user_id','movie_id','rating','timestamp']
mratings.columns = columns
rating_ugrp = mratings.groupby('user_id')['rating']
rating_mgrp = mratings.groupby('movie_id')['rating']


Check some stats on the number of ratings given by users

In [ ]:
urating_cnt = rating_ugrp.count()
urating_cnt.describe()

Check some stats on the number of ratings received by movies

In [ ]:
mrating_cnt = rating_mgrp.count()
mrating_cnt.describe()

Check some stats on the ratings given by users

In [ ]:
urating_avg = rating_ugrp.mean()
urating_avg.describe()

Check some stats on the ratings received by movies

In [ ]:
mrating_avg = rating_mgrp.mean()
mrating_avg.describe()

Use the user_id and movie_id as indeces of the factorized matrix M = Users_emb x Movies_emd.T. Subract one to make them zero based.  
Do a random split of 80% training and 20% validation data.  
Make the ratings zero-mean by subtracting the mean computed on the training set.

In [ ]:
#get the indices for users and movies. Make is zero base since they start from 1
indices = mratings[['user_id','movie_id']].values - 1
#shuffle the indices to create train and validation sets
indx = np.arange(indices.shape[0])
np.random.shuffle(indx)
split_percent = 0.8
split_train = int(round(split_percent*indx.size))
indx_train = TT(indices[indx[:split_train]]).int()
indx_val = TT(indices[indx[-(indx.size - split_train):]]).int()
rate_train_orig = mratings['rating'].values.astype(float)[indx[:split_train]]
rate_train = TT(rate_train_orig)
avg_rate = rate_train.mean()
rate_train -= avg_rate
rate_val_orig = mratings['rating'].values.astype(float)[indx[-(indx.size - split_train):]]
rate_val = TT(rate_val_orig)
rate_val -= avg_rate#remove the train mean

Perform a sanity check to make sure that the indeces used for training correspond to the correct original position.

In [ ]:
#sanity check
all_good = True
for _ in range(1000):
    #get any original index
    i = np.random.randint(len(indx_train))
    #get the corresponding index in the training set
    indx_now = indx_train[i].numpy()
    #check that the ratings in the training set and original are the same 
    if rate_train_orig[i] != mratings.loc[(mratings['user_id'] - 1 == indx_now[0]) & (mratings['movie_id'] - 1 == indx_now[1])]['rating'].values[0]:
        all_good = False
all_good

Create the loss and the model. For the loss, beside the mean square error between prediction and truth, we add a *L2* regularization term to ensure that the elements of the embedding are not too large and another regurarization term called *gravity* to make sure that the ratings predicted are not too large.  
The model simply returns the user and the movie embeddings. 

In [ ]:
class CFLoss(nn.Module):
    def __init__(self,lambda_g,lambda_r):
        super().__init__()
        self.lambda_g = lambda_g
        self.lambda_r = lambda_r
    def forward(self,ratings,iuemb,imemb,indices_um,print_l=False):
        uemb = iuemb.weight
        memb = imemb.weight
        approx = torch.matmul(uemb,memb.T)
        #MSE loss
        loss = ((approx[indices_um[:,0],indices_um[:,1]] - ratings)*(approx[indices_um[:,0],indices_um[:,1]] - ratings)).mean()
        #L2 regularization loss 
        lossr = self.lambda_r*((uemb*uemb).sum()/uemb.shape[0] + (memb*memb).sum()/memb.shape[0])
        #Gravity loss 1/(num_users*num_movies)sum_{i,j}(<uemb_i,memb_j>**2)
        lossg = self.lambda_g*(approx*approx).sum()/(uemb.shape[0]*memb.shape[0])
        if print_l:
            print('Losses',loss.item(),lossr.item(),lossg.item())
        return loss + lossr + lossg

#Collaborative FilteringModel
class CFModel(nn.Module):
    def __init__(self, num_users, num_movies, emb_size):
        super().__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.movie_emb = nn.Embedding(num_movies, emb_size)
    
    def forward(self):
        return self.user_emb,self.movie_emb
#Test loss
# nemb = 100
# uemb = nn.Embedding(nusers,nemb)
# memb = nn.Embedding(nmovies,nemb)
# lambda_g = 0.1
# lambda_r = 0.1
# cfloss = CFLoss(lambda_g,lambda_r)
# cfloss(rate_train,uemb,memb,indx_train)
        
        

Train the model using AdamW

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_users,num_movies = urating_avg.shape[0],mrating_avg.shape[0]
emb_size = 100
model = CFModel(num_users,num_movies,emb_size).to(device)
lr = 0.1
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lambda_g = 0.1
lambda_r = 0.1
cfloss = CFLoss(lambda_g,lambda_r)

nepochs = 20000
print_every = 200
rate_train_d = rate_train.to(device)
indx_train_d = indx_train.to(device)
rate_val_d = rate_val.to(device)
indx_val_d = indx_val.to(device)
model.train(True)
for j in range(nepochs):
    running_loss = 0.
    last_loss = 0.
    # Zero your gradients for every batch!
    optimizer.zero_grad()

    # Make predictions for this batch
    uemb,memb = model()

    # Compute the loss and its gradients
    loss = cfloss(rate_train_d,uemb,memb,indx_train_d).to(device)
    loss.backward()

    # Adjust learning weights
    optimizer.step()

    # Gather data and report
    last_loss = loss.item()
    if j % print_every == 0:
        model.eval()
        # Disable gradient computation and reduce memory consumption.
        with torch.no_grad():
            uemb,memb = model()
            loss = cfloss(rate_val_d,uemb,memb,indx_val_d).to(device)
            print(f'epoch {j} train loss: {last_loss}, validation loss: {loss.item()}')

        model.train(True)
        # tb_x = epoch_index * len(training_loader) + i + 1
        # tb_writer.add_scalar('Loss/train', last_loss, tb_x)

In [ ]:
#Read in the list of movies with title and genre colums 5 to 23 with 1 if of that genre
df_item = pd.read_csv('/home/giangi/Workspace/Data/ml-100k/u.item',sep='|',header=None,encoding='latin-1')
#Read the mapping of the columns 5 to 25 to genre name
df_genre_str = pd.read_csv('/home/giangi/Workspace/Data/ml-100k/u.genre',sep='|',header=None,encoding='latin-1')

In [ ]:
df_item

In [ ]:
df_genre_str

Movie might have multiple categories (genres), so just get one as representative.

In [ ]:
sel = [1]
sel.extend(list(range(5,24)))
i_genre = list(range(5,24))
#select only the movie name and the gernes from df_item
df_genre = df_item[sel]
#get just one of the genre per move. Get the underlaying matrix of just movie genre
dt_genre = df_genre[i_genre].values
#get the index sorting along genre. The ones will always be last so get the index of last element
col_gen = dt_genre.argsort(1)[:,-1]
#Make sure only ones were selected
np.unique(dt_genre[np.arange(len(col_gen)),col_gen])

In [ ]:
df_genre

In [ ]:
df_genre_str

col_gen cotains the numerical value of the genre which is also the index in df_genre_str. Select for each movie the genre name and numerical value from df_genre_str and concatenate the actual name of the movie

In [ ]:

df_movie_genre = pd.concat([df_genre_str.loc[col_gen].reset_index(drop=True),df_genre[1]],axis=1,ignore_index=True)
df_movie_genre

Group by the numerical values of the genre and get the mapping of each group, i.e. for each numerical genre which movie_ids belong to it. Print how many movies belong to each genre. Note that we assumed that a movie belong to only genre.


In [ ]:
movie_indx = df_movie_genre.groupby(1).indices
for k,v in movie_indx.items():
    print(k,len(v),df_genre_str.loc[k].values)

Use tsne to do dimentionality reduction of the movies embeddings. One might expect that different genres are further apart in the embedding space.

In [ ]:
from sklearn.manifold import TSNE
X = model.movie_emb.weight.detach().cpu().numpy()
X_embedded = TSNE(n_components=3, learning_rate='auto',init='random', perplexity=3).fit_transform(X)

Pick two genres that should not overlap like *Documentary* and *Horror* (7,11) or *Comedy* and *Drama* (5,8). Something like *Action* and *Adveture* might have both labels.  
Rotate 3D figure to see if there is a view angle that shows one color more than the other (grouping)

In [ ]:
plt.close('all')
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
mark = ['o','^']
which = [5,8]
i = 0
for (k,v) in movie_indx.items():
    if k in which:
        ax.scatter(X_embedded[v,0],X_embedded[v,1],X_embedded[v,2],mark[i])
        i += 1

Some sanity checks.

In [ ]:
#Some manual verification to make sure the results are correct
iuemb,imemb = model()
uemb = iuemb.weight
memb = imemb.weight
#get the factorized matrix
approx = torch.matmul(uemb,memb.T).detach().cpu().numpy()

In [ ]:
#from the validation set get the ones  whose predicted rating and < .5 from the truth
sel = np.where(np.abs(approx[indx_val[:,0],indx_val[:,1]] - rate_val.numpy()) < .5)
indx_good = indx_val[sel].numpy()

In [ ]:
#check that the difference is less than .5. Need to add back the avg_rate to compare
np.abs(mratings.loc[indx[-(indx.size - split_train):][sel]]['rating'] - (approx[indx_good[:,0],indx_good[:,1]] + avg_rate.numpy())).max()